In [2]:
import pandas as pd
!pip install xlrd
!pip install openpyxl

In [65]:
gdp_df = pd.read_csv('datasets/GDP1.csv')
### Next line unpivots table to have each year on a single row
gdp_df = pd.melt(gdp_df, id_vars='Country Name', value_vars=list(gdp_df.columns[4:]), var_name='Year', value_name='GDP')
gdp_df = gdp_df.rename(columns={'Country Name': 'Country'})
gdp_df['Year'] = gdp_df['Year'].astype('int64')

In [66]:
#contains happiness scores. From World Happiness Report
happy_data = pd.DataFrame()

happy_data = pd.read_csv('datasets/Final_Happiness_Data.csv', encoding='latin-1')
#year_df['Year'] = year
happy_data = year_df[['Country','Happiness Score',
   'Year']]

In [67]:
# contains all other info about how happiness score was derived. From World Happiness Report
happy_data1 = pd.read_excel('datasets/DataForTable2.1.xls')
happy_data1 = happy_data1.rename(columns={'Country name': 'Country', 'year': 'Year'})

In [68]:
happy_df = pd.merge(happy_data1, happy_data, on=['Country', 'Year'], how='left')#left_on=["Country name", 'year'], right_on=['Country', 'Year'], how='left')

In [69]:
le_df = pd.read_csv('datasets/Life_Expectancy.csv')
### Next line unpivots table to have each year on a single row
le_df = pd.melt(le_df, id_vars='Country Name', value_vars=list(le_df.columns[4:]), var_name='Year', value_name='Life Expectancy')
le_df = le_df.rename(columns={'Country Name': 'Country'})
le_df['Year'] = le_df['Year'].astype('int64')

In [70]:
xlsx = pd.ExcelFile('datasets/Freedom.xlsx')
freedom_df = pd.read_excel(xlsx, 'FIW13-22', header=1)
freedom_df = freedom_df.rename(columns={'Country/Territory': 'Country', 'Edition': 'Year', 'Total': 'Freedom_Rating'})
freedom_df = freedom_df[['Country', 'Year', 'Freedom_Rating']]

In [71]:
# merge all dfs
df = happy_df.merge(freedom_df, on=['Country', 'Year'], how='left')
df = df.merge(le_df, on=['Country', 'Year'], how='left')
df = df.merge(gdp_df, on=['Country', 'Year'], how='left')

In [73]:
df.to_csv('all_data.csv')

In [72]:
df

,Country,Year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect,Confidence in national government,Happiness Score,Freedom_Rating,Life Expectancy,GDP
0,Afghanistan,2008,3.723590,7.302574,0.450662,50.500000,0.718114,0.173169,0.881686,0.414297,0.258195,0.612072,3.723590,NaN,59.930,1.010931e+10
1,Afghanistan,2009,4.401778,7.472446,0.552308,50.799999,0.678896,0.195469,0.850035,0.481421,0.237092,0.611545,4.401778,NaN,60.484,1.241616e+10
2,Afghanistan,2010,4.758381,7.579183,0.539075,51.099998,0.600127,0.125859,0.706766,0.516907,0.275324,0.299357,4.758381,NaN,61.028,1.585668e+10
3,Afghanistan,2011,3.831719,7.552006,0.521104,51.400002,0.495901,0.167723,0.731109,0.479835,0.267175,0.307386,3.831719,NaN,61.553,1.780511e+10
4,Afghanistan,2012,3.782938,7.637953,0.520637,51.700001,0.530935,0.241247,0.775620,0.613513,0.267919,0.435440,3.782938,NaN,62.054,1.990732e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2084,Zimbabwe,2017,3.638300,8.241609,0.754147,52.150002,0.752826,-0.113937,0.751208,0.733641,0.224051,0.682647,3.638300,32.0,60.812,1.758489e+10
2085,Zimbabwe,2018,3.616480,8.274620,0.775388,52.625000,0.762675,-0.084747,0.844209,0.657524,0.211726,0.550508,3.616480,30.0,61.195,1.811554e+10
2086,Zimbabwe,2019,2.693523,8.196998,0.759162,53.099998,0.631908,-0.081540,0.830652,0.658434,0.235354,0.456455,2.693523,31.0,61.490,1.928429e+10
2087,Zimbabwe,2020,3.159802,8.117733,0.717243,53.575001,0.643303,-0.029376,0.788523,0.660658,0.345736,0.577302,3.159802,29.0,61.738,1.805117e+10
